# 🌍 Microsoft Agent Framework (.NET) के साथ AI ट्रैवल एजेंट

## 📋 परिदृश्य का अवलोकन

यह नोटबुक Microsoft Agent Framework for .NET का उपयोग करके एक बुद्धिमान यात्रा योजना एजेंट बनाने का तरीका दिखाता है। यह एजेंट दुनिया भर के रैंडम डेस्टिनेशन के लिए व्यक्तिगत दिन-यात्रा योजनाएं स्वचालित रूप से तैयार कर सकता है।

**मुख्य क्षमताएं:**
- 🎲 **रैंडम डेस्टिनेशन चयन**: छुट्टी के स्थान चुनने के लिए कस्टम टूल का उपयोग करता है
- 🗺️ **बुद्धिमान यात्रा योजना**: विस्तृत दिन-प्रतिदिन की योजनाएं बनाता है
- 🔄 **रियल-टाइम स्ट्रीमिंग**: तत्काल और स्ट्रीमिंग प्रतिक्रियाओं का समर्थन करता है
- 🛠️ **कस्टम टूल इंटीग्रेशन**: एजेंट की क्षमताओं को बढ़ाने का तरीका दिखाता है

## 🔧 तकनीकी संरचना

### मुख्य तकनीकें
- **Microsoft Agent Framework**: AI एजेंट विकास के लिए नवीनतम .NET कार्यान्वयन
- **GitHub Models Integration**: GitHub के AI मॉडल इनफरेंस सेवा का उपयोग करता है
- **OpenAI API Compatibility**: कस्टम एंडपॉइंट्स के साथ OpenAI क्लाइंट लाइब्रेरी का लाभ उठाता है
- **सुरक्षित कॉन्फ़िगरेशन**: पर्यावरण-आधारित API कुंजी प्रबंधन

### मुख्य घटक
1. **AIAgent**: मुख्य एजेंट ऑर्केस्ट्रेटर जो बातचीत के प्रवाह को संभालता है
2. **कस्टम टूल्स**: `GetRandomDestination()` फ़ंक्शन एजेंट के लिए उपलब्ध है
3. **चैट क्लाइंट**: GitHub Models-समर्थित बातचीत इंटरफ़ेस
4. **स्ट्रीमिंग समर्थन**: रियल-टाइम प्रतिक्रिया उत्पन्न करने की क्षमता

### इंटीग्रेशन पैटर्न
```mermaid
graph LR
    A[User Request] --> B[AI Agent]
    B --> C[GitHub Models API]
    B --> D[GetRandomDestination Tool]
    C --> E[Travel Itinerary]
    D --> E
```

## 🚀 आरंभ करना

**पूर्वापेक्षाएँ:**
- .NET 9.0 या उच्चतर
- GitHub Models API एक्सेस टोकन
- `.env` फ़ाइल में कॉन्फ़िगर किए गए पर्यावरण चर

**आवश्यक पर्यावरण चर:**
```env
GITHUB_TOKEN=your_github_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

नीचे दिए गए सेल्स को क्रम में चलाएं और ट्रैवल एजेंट को क्रियान्वित होते हुए देखें!


In [1]:
// 📦 NuGet Package Reference: Microsoft Extensions AI
// This provides the core AI abstraction layer for .NET applications
// Essential for building AI agents with standardized interfaces
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
// 🔗 Local Reference: OpenAI Agent Extensions
// This references the local build of the Microsoft Extensions AI Agents OpenAI library
// Contains OpenAI-specific implementations for the agent framework
// Required for integrating with OpenAI models and GitHub Models
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:

#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// 🔌 NuGet Package Reference: OpenAI Integration
// Preview version of Microsoft Extensions AI OpenAI integration
// Provides OpenAI client and service implementations
// Enables communication with OpenAI API and GitHub Models
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.1-preview.1.25474.6"

In [5]:
// 🔧 NuGet Package Reference: Environment Variables
// DotNetEnv allows loading environment variables from .env files
// Essential for managing API keys and configuration securely
// Helps keep sensitive information out of source code
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
// 📚 Import Required Namespaces
using System;                          // Core system functionality and base types
using System.ComponentModel;           // For Description attributes on tool functions
using System.ClientModel;             // Client model abstractions for API interactions

using Microsoft.Extensions.AI;        // Core AI abstractions and interfaces
using Microsoft.Agents.AI;           // Agent framework classes and interfaces
using OpenAI;                         // OpenAI client library for API communication

In [7]:
// 🔧 Import Environment Variable Loading
// DotNetEnv namespace for loading .env files
// Enables secure configuration management from external files
using DotNetEnv;

In [8]:
// 🔧 Load Environment Variables from .env File
// This loads configuration from a .env file located 3 directories up
// Required variables: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
// Ensures secure handling of API credentials and configuration
Env.Load("../../../.env");

In [9]:
// 🎲 Tool Function: Random Destination Generator
// This static method will be available to the agent as a callable tool
// The [Description] attribute helps the AI understand when to use this function
// This demonstrates how to create custom tools for AI agents
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    // List of popular vacation destinations around the world
    // The agent will randomly select from these options
    var destinations = new List<string>
    {
        "Paris, France",         // European cultural capital
        "Tokyo, Japan",          // Asian modern metropolis
        "New York City, USA",    // American urban experience
        "Sydney, Australia",     // Oceanic coastal beauty
        "Rome, Italy",           // Historical European city
        "Barcelona, Spain",      // Mediterranean cultural hub
        "Cape Town, South Africa", // African scenic destination
        "Rio de Janeiro, Brazil",  // South American beach city
        "Bangkok, Thailand",        // Southeast Asian cultural center
        "Vancouver, Canada"         // North American natural beauty
    };

    // Generate random index and return selected destination
    // Uses System.Random for simple random selection
    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
// 🔑 Extract Configuration from Environment Variables
// Retrieve GitHub Models API endpoint, throws exception if not set
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
// Get the model ID, defaults to gpt-4o-mini if not specified
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
// Retrieve GitHub token for authentication, throws exception if not set
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
// ⚙️ Configure OpenAI Client Options
// Create configuration options to point to GitHub Models endpoint
// This redirects OpenAI client calls to GitHub's model inference service
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)  // Set custom endpoint for GitHub Models
};

In [12]:
// 🔌 Initialize OpenAI Client with GitHub Models Configuration
// Create OpenAI client using GitHub token for authentication
// Configure it to use GitHub Models endpoint instead of OpenAI directly
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
// 🤖 Create AI Agent with Travel Planning Capabilities
// Initialize OpenAI client, get chat client for specified model, and create AI agent
// Configure agent with travel planning instructions and random destination tool
// The agent can now plan trips using the GetRandomDestination function
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions)
    .GetChatClient(github_model_id)  // Get chat client for the specified model
    .CreateAIAgent(
        instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", 
        tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]  // Register the tool function
    );

In [14]:
// 🚀 Execute Agent: Plan a Day Trip (Non-Streaming)
// Run the agent with a travel planning request
// The agent will use the GetRandomDestination tool and create a complete itinerary
// Returns the full response once processing is complete
Console.WriteLine(await agent.RunAsync("Plan me a day trip"));

For your day trip to Tokyo, Japan, here's a suggested itinerary:

### Morning
- **Breakfast**: Start your day with a traditional Japanese breakfast. You can try grilled fish, rice, and miso soup at a local restaurant or enjoy a quick meal at a convenience store.
- **Visit Senso-ji Temple**: Head to Asakusa and visit Tokyo's oldest temple, Senso-ji. Explore the Nakamise shopping street leading up to the temple for snacks and souvenirs.

### Mid-Morning
- **Explore Ueno Park**: After Senso-ji, take a short train ride to Ueno Park. Stroll through the park, visit the Ueno Zoo, or check out one of the many museums like the Tokyo National Museum.

### Lunch
- **Try Sushi**: For lunch, indulge in fresh sushi at a conveyor belt sushi restaurant or a well-reviewed local sushi place in the area.

### Afternoon
- **Visit Akihabara**: Spend your afternoon exploring Akihabara, a district famous for electronics and otaku culture. Visit some anime and manga stores, or check out a themed cafe.
- **Exp

In [15]:
// 🌊 Execute Agent: Plan a Day Trip (Streaming Response)
// Run the agent with streaming enabled for real-time response display
// Shows the agent's thinking and response as it generates the content
// Provides better user experience with immediate feedback
await foreach (var update in agent.RunStreamingAsync("Plan me a day trip"))
{
    Console.Write(update);  // Display each part of the response as it arrives
}

How about a day trip to Vancouver, Canada? Here’s a suggested itinerary for your day:

### Morning:
- **Breakfast at a Local Café**: Start your day at a cozy café in Gastown. Try the famous Canadian pancakes or a classic bagel with smoked salmon.
- **Stanley Park**: After breakfast, head to Stanley Park. Enjoy a walk or rent a bike to explore the scenic seawall, lush gardens, and totem poles.

### Afternoon:
- **Granville Island**: Visit Granville Island for lunch. Explore the public market, artisan shops, and try some delicious local food from the various food stalls.
- **Aquabus Ride**: Take an Aquabus ride across False Creek to enjoy stunning views of the city and the North Shore mountains.

### Evening:
- **Capilano Suspension Bridge**: After lunch, visit the Capilano Suspension Bridge Park. Walk across the bridge and enjoy the beautiful forest scenery.
- **Dinner at a Waterfront Restaurant**: Finish the day with dinner at a waterfront restaurant, where you can enjoy fresh seafood 


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल दस्तावेज़, जो इसकी मूल भाषा में है, को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
